In [ ]:
import pandas as pd
import multiprocessing as mp
import numpy as np
from tqdm import tqdm

logs = pd.read_csv("./data_nov/logs_nov.csv")
proc = pd.read_csv("./data_nov/processes_nov.csv")
orders = pd.read_csv("./data_nov/orders_nov.csv")
logs.time_out = pd.to_datetime(logs.time_out, infer_datetime_format=True)
orders.set_index('order_number', inplace=True)

In [ ]:
vip_orders = orders[(~orders.product_length.isnull()) & (orders.order_status == 55) & orders.reference_number.str.startswith('HK') & (orders.item_description.str.contains('RAIL'))].copy()
vip_orders['y'] = 0
vip_orders['total_steps'] = 0
vip_orders['cur_step'] = 0
vip_mos = vip_orders.index.values.tolist()
vip_logs = logs[(logs.order_number.isin(vip_mos)) & (logs.quantity_complete != 0)].copy()
vip_logs.sort_values('sequence_code', ascending=True, inplace=True)
drop_cols = ['customer_job_number', 'reference_number', 'item_number', 'product_variant', 'product_clearance',  'product_block_count', 'product_g1', 'product_pitch',  'order_status', 'order_release_code', 'order_customer_name', 'order_replied_due', 'order_scheduled_due', 'order_original_due', 'order_scheduled_shipping_date', 'order_class', 'promise_date', 'sales_item_number', 'printed_due']
vip_orders = vip_orders.drop(columns=drop_cols)
vip_logs['time_out'] = pd.to_datetime(vip_logs['time_out'])


In [ ]:
combined = ['item_description','product_family', 'product_model', 'product_length',
'order_quantity', 'D0010', 'D0020', 'F0010', 'C0010',
'C0020', 'C0060', 'C0080', 'C0090', 'K0010', 'K0020', 'B0010', 'B0020',
'B0021', 'B0025', 'B0060', 'B0080', 'C0075', 'J0020', 'K0011', 'C0070',
'C0100', 'C0110', 'C0120', 'H0010', 'H0020', 'H0030', 'H0070', 'J0050',
'J0060', 'B0012', 'B0040', 'B0070', 'C0040', 'B0062', 'C0050', 'C0030',
'C0031', 'C0035', 'H0046', 'GL010', 'GL130', 'GL140', 'B0050', 'H0045',
'H0060', 'H0050', 'B0030', 'F0020', 'J0025', 'B0015', 'GL030', 'H0051', 'total_steps', 'cur_step', 'y']

In [ ]:
THROTTLE_MIN = 1800
vip_mos = vip_orders.index.values.tolist()
off_days = ['2021-09-04', '2021-09-05', '2021-09-11', '2021-09-12', '2021-09-18', '2021-09-19', '2021-09-25', '2021-09-26', '2021-10-02', '2021-10-03', '2021-10-09', '2021-10-10', '2021-10-16', '2021-10-17', '2021-10-23', '2021-10-24', '2021-10-30', '2021-10-31', '2021-11-06', '2021-11-07', '2021-11-13', '2021-11-14', '2021-11-20', '2021-11-21', '2021-11-25', '2021-11-26', '2021-11-27', '2021-11-28', '2021-12-04', '2021-12-05', '2021-12-11', '2021-12-12', '2021-12-18', '2021-12-19', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26', '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-08', '2022-01-09', '2022-01-15', '2022-01-16', '2022-01-22', '2022-01-23', '2022-01-29', '2022-01-30', '2022-02-05', '2022-02-06', '2022-02-12', '2022-02-13', '2022-02-19', '2022-02-20', '2022-02-26', '2022-02-27', '2022-03-05', '2022-03-06', '2022-03-12', '2022-03-13', '2022-03-19', '2022-03-20', '2022-03-26', '2022-03-27', '2022-04-02', '2022-04-03', '2022-04-09', '2022-04-10', '2022-04-16', '2022-04-17', '2022-04-23', '2022-04-24', '2022-04-30', '2022-05-01', '2022-05-07', '2022-05-08', '2022-05-14', '2022-05-15', '2022-05-21', '2022-05-22', '2022-05-28', '2022-05-29', '2022-05-30', '2022-06-04', '2022-06-05', '2022-06-11', '2022-06-12', '2022-06-18', '2022-06-19', '2022-06-25', '2022-06-26', '2022-07-02', '2022-07-03', '2022-07-04', '2022-07-05', '2022-07-09', '2022-07-10', '2022-07-16', '2022-07-17', '2022-07-23', '2022-07-24', '2022-07-30', '2022-07-31', '2022-08-06', '2022-08-07', '2022-08-13', '2022-08-14', '2022-08-20', '2022-08-21', '2022-08-27', '2022-08-28', '2022-09-03', '2022-09-04', '2022-09-05', '2022-09-10', '2022-09-11', '2022-09-17', '2022-09-18', '2022-09-24', '2022-09-25', '2022-10-01', '2022-10-02', '2022-10-08', '2022-10-09', '2022-10-15', '2022-10-16', '2022-10-22', '2022-10-23', '2022-10-29', '2022-10-30', '2022-11-05', '2022-11-06']
o_days = pd.to_datetime(off_days, infer_datetime_format=True, utc=True)
answer = pd.DataFrame(index=range(224592),columns=combined)
ind = 0
def f(mo):      
    mo_logs = vip_logs[vip_logs.order_number == mo].sort_values('sequence_code')    
    mo_logs = mo_logs.drop_duplicates(subset=['sequence_code'], keep='first')    
    mo_logs = mo_logs.drop_duplicates(subset=['facility_id'], keep='first')    
    mo_logs['start'] = mo_logs['time_out'].shift(1)
    mo_logs['diff'] = mo_logs['time_out'].diff(periods=1).astype('timedelta64[m]')                
    mo_logs['wcount'] = 0
    mo_logs['diff2'] = 0
    
    if len(mo_logs[mo_logs['diff'] < 0]):
        print('negative', mo)
        return
    try:                    
        for index, start, time_out in zip(mo_logs.index.tolist(), mo_logs['start'], mo_logs['time_out']):                                
                mo_logs.loc[index, 'wcount'] = len(list(x for x in o_days if (start < x < time_out)))        
        mo_logs['diff2'] = mo_logs['diff'] - mo_logs['wcount'] * 1440        
        mo_logs.loc[mo_logs['diff2'] > THROTTLE_MIN, 'diff2'] = THROTTLE_MIN                
        facilites = mo_logs['facility_id'].values[1:]
        durations = mo_logs['diff2'].values[1:]                
        return mo, facilites, durations            
    except Exception as e: print(mo, e)

def append_res(result):      
    global ind          
    for i in result:
        if i:      
            mo, facilites, durations = i      
            try:                                               
                vip_orders.loc[mo, 'total_steps'] = len(facilites)
                cur_step = 0
                acc_duration = 0                
                for facility, duration in zip(facilites, durations):                    
                    vip_orders.loc[mo, facility] = 1
                    cur_step += 1
                    acc_duration += duration
                    vip_orders.loc[mo, 'cur_step'] = cur_step
                    vip_orders.loc[mo, 'y'] = acc_duration            
                    answer.loc[ind] = vip_orders.loc[mo].copy()
                    ind += 1                
            except Exception as e: print(e)

if __name__ == "__main__":        
    with mp.Pool(16) as pool:        
        print('pool started')
        results = pool.map_async(f, vip_mos, callback = append_res)
        # results = pool.map_async(f, vip_mos)
        results.wait() 
print("done")
answer.describe()

In [ ]:
answer = answer[~answer['y'].isnull()]
len(answer)

In [19]:
test = answer.copy()
test['ZLE'] = np.where(test['item_description'].str.count('LE'), 1, 0)
test['ZGK'] = np.where(test['item_description'].str.count('GK'), 1, 0)
test['ZLT'] = np.where(test['item_description'].str.count('LT'), 1, 0)
test['ZGP'] = np.where(test['item_description'].str.count('GP'), 1, 0)
test['product_length'] = test['product_length'] / 1000
test['y'] = test['y'] / 60
test.fillna(0, inplace=True)
dimension = len(test.columns) - 1


In [20]:
list_classify = ["product_family", "product_model"]
for i in list_classify:    
    test[i], uniques = pd.factorize(test[i], sort=True)    
    print(uniques)
test = test[combined[1:]]
shuffled = test.sample(frac=1)
pivot = int(len(test) * .1)
train = shuffled.iloc[pivot:].copy()
val = shuffled.iloc[:pivot].copy()
train = train.astype(float)
val = val.astype(float)

Index([0, 'HCR', 'HDR', 'HRW', 'HSR', 'SHS', 'SHW', 'SR', 'SRS', 'SRS-W'], dtype='object')
Index([        0,      '12',      '15', '15/300R',      '17',      '20',
            '21',      '25',      '27',      '30',      '35',      '45',
       '45/800R',      '55',      '65',       '9',      '9X'],
      dtype='object')


In [22]:
len(test.columns)

59

In [23]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

class MyDataset(Dataset):   
  def __init__(self,data):     
    super(MyDataset).__init__()
    x=data.iloc[:,0:58].values
    y=data.iloc[:,58].values     
    self.x_train=torch.tensor(x,dtype=torch.float32).cuda()    
    self.y_train=torch.tensor(y,dtype=torch.float32).view(-1, 1).cuda()    
 
  def __len__(self):
    return len(self.y_train)   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]
 
  def __len__(self):
    return len(self.x_train)   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

train_set=MyDataset(train)
val_set=MyDataset(val)
train_dl=DataLoader(train_set, batch_size=256, shuffle=True, num_workers=0)
val_dl=DataLoader(val_set, batch_size=len(val_set))

In [24]:
loss_fn = F.mse_loss
def fit(num_epochs, model, loss_fn, opt):
    val_best  = 26
    for epoch in range(num_epochs):        
        for xb,yb in train_dl:            
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        if epoch % 10 == 0:
            loss_val = loss.item()
            print(loss_val, epoch)            
            for v_xb,v_yb in val_dl:
                result = model(v_xb)     
                val_err = torch.mean(abs(v_yb - result) / v_yb).item() * 100                    
                print("val_err: ", val_err)
            if val_err < val_best:
                val_best = val_err
                torch.save(model.state_dict(), f"throttle_{str(val_err).replace('.', '_')[:5]}.pth")
    
class SimpleNet(nn.Module):    
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(58, 256)                
        self.linear2 = nn.Linear(256, 58)        
        self.linear3 = nn.Linear(58, 1)
        
        self.act1 = nn.ReLU() # Activation function        
        self.act2 = nn.ReLU() # Activation function        
        
            
    # Perform the computation
    def forward(self, x):                
        x1 = self.linear1(x)
        x1_act = self.act1(x1)
        x2 = self.linear2(x1_act)
        x2_act = self.act2(x2) + x
        x3 = self.linear3(x2_act)
        return x3

model = SimpleNet().cuda()
opt = torch.optim.SGD(model.parameters(), 1e-6, momentum=0.9)
fit(1000, model, loss_fn, opt)

588.0640258789062 0
val_err:  127.95497179031372
448.9702453613281 10
val_err:  85.48399806022644
316.9205322265625 20
val_err:  78.45739126205444
312.27288818359375 30
val_err:  78.29025983810425
263.7007141113281 40
val_err:  77.92910933494568
347.0400085449219 50
val_err:  80.0000250339508
481.3423767089844 60
val_err:  77.35064625740051
356.8951721191406 70
val_err:  78.71100306510925
348.1722717285156 80
val_err:  78.00238728523254
261.3226318359375 90
val_err:  78.51866483688354
265.8144836425781 100
val_err:  75.52003264427185
422.6971130371094 110
val_err:  78.52603793144226
386.86175537109375 120
val_err:  74.41136837005615
399.2859191894531 130
val_err:  79.65936660766602
317.7434997558594 140
val_err:  77.54520773887634
278.7809753417969 150
val_err:  74.32677745819092
302.62298583984375 160
val_err:  75.46496391296387
287.3844299316406 170
val_err:  79.89340424537659
243.50186157226562 180
val_err:  82.28708505630493
346.1545104980469 190
val_err:  76.24913454055786
414.139

KeyboardInterrupt: 

In [ ]:
len(test)

In [ ]:
# low_days = ['2021-07-03', '2021-07-04', '2021-07-10', '2021-07-11', '2021-07-17', '2021-07-18', '2021-07-24', '2021-07-25', '2021-07-31', '2021-08-01', '2021-08-07', '2021-08-08', '2021-08-14', '2021-08-15', '2021-08-21', '2021-08-22', '2021-08-28', '2021-08-29', '2021-09-04', '2021-09-05', '2021-09-11', '2021-09-12', '2021-09-18', '2021-09-19', '2021-09-25', '2021-09-26', '2021-10-02', '2021-10-03', '2021-10-09', '2021-10-10', '2021-10-16', '2021-10-17', '2021-10-23', '2021-10-24', '2021-10-30', '2021-10-31', '2021-11-06', '2021-11-07', '2021-11-13', '2021-11-14', '2021-11-20', '2021-11-21', '2021-11-27', '2021-11-28', '2021-12-04', '2021-12-05', '2021-12-11', '2021-12-12', '2021-12-18', '2021-12-19', '2021-12-25', '2021-12-26', '2021/06/05', '2021/06/06', '2021/06/13', '2021/06/19', '2021/06/20', '2021/06/27', '2021/06/30', '2021/07/02', '2021/07/03', '2021/07/04', '2021/07/05', '2021/07/11', '2021/07/17', '2021/07/18', '2021/07/25', '2021/07/31', '2021/08/01', '2021/08/08', '2021/08/14', '2021/08/15', '2021/08/22', '2021/08/28', '2021/08/29', '2021/09/04', '2021/09/05', '2021/09/06', '2021/09/12', '2021/09/18', '2021/09/19', '2021/09/26', '2021/10/02', '2021/10/03', '2021/10/10', '2021/10/16', '2021/10/17', '2021/10/24', '2021/10/30', '2021/10/31', '2021/11/07', '2021/11/13', '2021/11/14', '2021/11/21', '2021/11/25', '2021/11/26', '2021/11/27', '2021/11/28', '2021/12/05', '2021/12/11', '2021/12/12', '2021/12/19', '2021/12/23', '2021/12/24', '2021/12/25', '2021/12/26', '2021/12/31', '2022-01-01', '2022-01-02', '2022-01-08', '2022-01-09', '2022-01-15', '2022-01-16', '2022-01-22', '2022-01-23', '2022-01-29', '2022-01-30', '2022-02-05', '2022-02-06', '2022-02-12', '2022-02-13', '2022-02-19', '2022-02-20', '2022-02-26', '2022-02-27', '2022-03-05', '2022-03-06', '2022-03-12', '2022-03-13', '2022-03-19', '2022-03-20', '2022-03-26', '2022-03-27', '2022-04-02', '2022-04-03', '2022-04-09', '2022-04-10', '2022-04-16', '2022-04-17', '2022-04-23', '2022-04-24', '2022-04-30', '2022-05-01', '2022-05-07', '2022-05-08', '2022-05-14', '2022-05-15', '2022-05-21', '2022-05-22', '2022-05-28', '2022-05-29', '2022-06-04', '2022-06-05', '2022-06-11', '2022-06-12', '2022-06-18', '2022-06-19', '2022-06-25', '2022-06-26', '2022-07-02', '2022-07-03', '2022-07-09', '2022-07-10', '2022-07-16', '2022-07-17', '2022-07-23', '2022-07-24', '2022-07-30', '2022-07-31', '2022-08-06', '2022-08-07', '2022-08-13', '2022-08-14', '2022-08-20', '2022-08-21', '2022-08-27', '2022-08-28', '2022-09-03', '2022-09-04', '2022-09-10', '2022-09-11', '2022-09-17', '2022-09-18', '2022-09-24', '2022-09-25', '2022-10-01', '2022-10-02', '2022-10-08', '2022-10-09', '2022-10-15', '2022-10-16', '2022-10-22', '2022-10-23', '2022-10-29', '2022-10-30', '2022-11-05', '2022-11-06', '2022/01/01', '2022/01/02', '2022/01/03', '2022/01/08', '2022/01/09', '2022/01/16', '2022/01/22', '2022/01/23', '2022/01/30', '2022/02/04', '2022/02/05', '2022/02/06', '2022/02/13', '2022/02/19', '2022/02/20', '2022/02/27', '2022/03/05', '2022/03/06', '2022/03/12', '2022/03/13', '2022/03/19', '2022/03/20', '2022/03/27', '2022/04/02', '2022/04/03', '2022/04/10', '2022/04/15', '2022/04/16', '2022/04/17', '2022/04/23', '2022/04/24', '2022/04/30', '2022/05/01', '2022/05/08', '2022/05/14', '2022/05/15', '2022/05/22', '2022/05/28', '2022/05/29', '2022/05/30', '2022/06/05', '2022/06/11', '2022/06/12', '2022/06/18', '2022/06/19', '2022/06/25', '2022/06/26', '2022/06/30', '2022/07/02', '2022/07/03', '2022/07/04', '2022/07/05', '2022/07/10', '2022/07/16', '2022/07/17', '2022/07/23', '2022/07/24', '2022/07/30', '2022/07/31', '2022/08/06', '2022/08/07', '2022/08/13', '2022/08/14', '2022/08/20', '2022/08/21', '2022/08/27', '2022/08/28', '2022/09/03', '2022/09/04', '2022/09/05', '2022/09/10', '2022/09/11', '2022/09/17', '2022/09/18', '2022/09/24', '2022/09/25', '2022/10/01', '2022/10/02', '2022/10/08', '2022/10/09', '2022/10/15', '2022/10/16', '2022/10/22', '2022/10/23', '2022/10/29', '2022/10/30', '2022/11/05', '2022/11/06']
# def f(mo):       
#     mo_logs = vip_logs[vip_logs.order_number == mo].copy()
#     mo_logs = mo_logs.drop_duplicates(subset=['sequence_code'], keep='first')
#     mo_logs['diff'] = mo_logs['time_out'].diff(periods=1).astype('timedelta64[m]')
#     mo_logs['start'] = mo_logs['time_out'].shift(1)    
#     mo_logs.loc[mo_logs['diff'] > THROTTLE_MIN, 'diff'] = THROTTLE_MIN
#     try:
#         # for index, row in mo_logs.iloc[1:].iterrows():                
#         #     if row['diff'] > THROTTLE_MIN:
#         #         end = row.time_out.strftime("%Y-%m-%d")
#         #         start = row.start.strftime("%Y-%m-%d")
#         #         offday_count = len(list(x for x in low_days if start < x < end))             
#         #         row['diff'] -= offday_count * 1440
#         #         if row['diff'] > THROTTLE_MIN:
#         #             # print(index)
#         #             mo_logs.loc[index, 'diff'] = THROTTLE_MIN    
#         period = mo_logs['diff'].sum() / 60
#         # return mo, period, mo_logs.iloc[0]['time_out']
#         return mo, period, len(mo_logs)
#     except Exception as e: print(mo, e)

# b_types = ['-III', '(GK)', '(GP)', '-II']
# p_types = ['K','E','P','F','T','S','M','H']
# for i in b_types:
#     vip_orders[i] = 0
# for i in p_types:
#     vip_orders[i] = 0
# vip_orders['custom'] = 0
# for index, row in vip_orders.iterrows():
#     try:
#         des = row['item_description'].split(' RAIL')[0].split('L')[1]
#         for bt in b_types:
#             if bt in des:
#                 vip_orders.loc[index, bt] = 1
#                 des = des.replace(bt, '')
#         print(des)
#         for pt in p_types:
#             if pt in des:            
#                 vip_orders.loc[index, pt] = 1
#     except Exception as e:
#         vip_orders.loc[index, 'custom'] = 1